In [8]:
import numpy as np
import pandas as pd
import json
import os

In [16]:
def predict_to_df(filepath):
    f = open(filepath,  encoding="utf8")
    label = json.load(f)
    columns = ['d', 's', 'e', 'key', 'sentences']
    dfs = []
    # df = pd.DataFrame(columns = ['d', 's', 'e', 'key', 'sentences'])
    sen_index = 0
    word_index = 0
    for sen in label:
        for word in sen['l']:
            d = word['d']
            s = word['s']
            e = word['e']
            dfs.append([d, s, e, d+str(word_index), sen_index])
            # df = df.append({'d':d, 's':s, 'e':e, 'key':d+str(word_index), 'sentences':sen_index}, ignore_index=True)
            word_index += 1
        sen_index += 1
    f.close()
    dfs = pd.DataFrame(dfs, columns = columns)
    return dfs

In [17]:
def label_to_df(filepath):
    f = open(filepath,  encoding="utf8")
    label = json.load(f)
    # df = pd.DataFrame(columns = ['d', 's', 'e', 'key', 'sentences'])
    columns = ['d', 's', 'e', 'key', 'sentences']
    dfs = []
    sen_index = 0
    word_index = 0
    for sen in label:
        for word in sen['l']:
            d = word['d']
            s = word['s']
            e = word['e']
            dfs.append([d, s, e, d+str(word_index), sen_index])
            # df = df.append({'d':d, 's':s, 'e':e, 'key':d+str(word_index), 'sentences':sen_index}, ignore_index=True)
            word_index += 1
        sen_index += 1
    f.close()
    dfs = pd.DataFrame(dfs, columns = columns)
    return dfs

In [7]:
def IoU(label_dir, predict_dir, filename):
#     config path file
    label_path = os.path.join(label_dir, filename + '.json')
    predict_path = os.path.join(predict_dir, filename + '.json')
#     read data
    df_predict = predict_to_df(predict_path)
    df_label = label_to_df(label_path)
    
#     calculate
    df_join = pd.merge(left=df_label, right=df_predict, on='key', how='left')
    df_join = df_join.fillna(0) #     fill na elements
    df_join['s_min'] = np.where((df_join['s_x'] >= df_join['s_y']), df_join['s_y'], df_join['s_x'])
    df_join['e_max'] = np.where((df_join['e_x'] >= df_join['e_y']), df_join['e_x'], df_join['e_y'])
    df_join['union'] = df_join['e_max'] - df_join['s_min']
    df_join['inter'] = df_join['union'] - np.abs(df_join['s_x']-df_join['s_y']) - np.abs(df_join['e_x']-df_join['e_y'])
    df_join['inter'] = np.where(df_join['inter']<0, 0, df_join['inter'])
#     add 1 to avoid error
    df_join['iou'] = np.round(df_join['inter']/(df_join['union']+1),2)
    
#     iou by sentence
    iou_by_sen = df_join.groupby(['sentences_x'])['iou'].mean()
#     iou total
    final_iou = iou_by_sen.mean()
    return iou_by_sen, final_iou

In [18]:
iou_segments = []

label_dir = '../data/train/labels/'
predict_dir = '../data/train/results/'

for file in os.listdir(label_dir):
    if file.endswith('.json'):
        filename = file.replace('.json', '')
        iou_sentence, iou_segment = IoU(label_dir, predict_dir, filename)
        iou_segments.append(iou_segment)
        
print(f'Final IoU: {np.mean(iou_segments)}')

Final IoU: 0.4099779003448909
